# Internal Dependencies
<br>  

### References
- [Analyze java package metrics in a graph database](https://joht.github.io/johtizen/data/2023/04/21/java-package-metrics-analysis.html)
- [Calculate metrics](https://101.jqassistant.org/calculate-metrics/index.html)
- [Neo4j Python Driver](https://neo4j.com/docs/api/python-driver/current)

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plot
from neo4j import GraphDatabase

In [2]:
# Please set the environment variable "NEO4J_INITIAL_PASSWORD" in your shell 
# before starting jupyter notebook to provide the password for the user "neo4j". 
# It is not recommended to hardcode the password into jupyter notebook for security reasons.

driver = GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j", os.environ.get("NEO4J_INITIAL_PASSWORD")))
driver.verify_connectivity()

In [3]:
def get_cypher_query_from_file(cypherFileName):
    with open(cypherFileName) as file:
        return ' '.join(file.readlines())

In [4]:
def query_cypher_to_data_frame(filename : str, limit: int = 10_000):
    cypher_query_template = "{query}\nLIMIT {row_limit}"
    cypher_query = get_cypher_query_from_file(filename)
    cypher_query = cypher_query_template.format(query = cypher_query, row_limit = limit)
    records, summary, keys = driver.execute_query(cypher_query)
    return pd.DataFrame([r.values() for r in records], columns=keys)

In [5]:
def query_first_non_empty_cypher_to_data_frame(*filenames : str, limit: int = 10_000):
    """
    Executes the Cypher queries of the given files and returns the first result that is not empty.
    If all given file names result in empty results, the last (empty) result will be returned.
    By additionally specifying "limit=" the "LIMIT" keyword will appended to query so that only the first results get returned.
    """    
    result=pd.DataFrame()
    for filename in filenames:
        result=query_cypher_to_data_frame(filename, limit)
        if not result.empty:
            return result
    return result

In [6]:
#The following cell uses the build-in %html "magic" to override the CSS style for tables to a much smaller size.
#This is especially needed for PDF export of tables with multiple columns.

In [7]:
%%html
<style>
/* CSS style for smaller dataframe tables. */
.dataframe th {
    font-size: 8px;
}
.dataframe td {
    font-size: 8px;
}
</style>

## Artifacts

List the artifacts this notebook is based on. Different sorting variations help finding artifacts by their features and support larger code bases where the list of all artifacts gets too long.

Only the top 30 entries are shown. The whole table can be found in the following CSV report:  
`List_all_Java_artifacts`

In [8]:
artifacts = query_cypher_to_data_frame("../cypher/Internal_Dependencies/List_all_Java_artifacts.cypher")

### Table 1a - Top 30 artifacts with the highest package count

In [9]:
# Sort by number of packages descending
artifacts.sort_values(by=['packages','artifactName'], ascending=[False, True]).reset_index(drop=True).head(30)

,artifactName,packages,types,incomingDependencies,outgoingDependencies
0,axon-messaging-4.10.0.jar,64,787,8,0
1,axon-server-connector-4.10.0.jar,11,131,1,4
2,axon-modelling-4.10.0.jar,10,156,6,1
3,axon-eventsourcing-4.10.0.jar,9,133,5,2
4,axon-spring-boot-autoconfigure-4.10.0.jar,9,74,0,7
5,axon-test-4.10.0.jar,8,87,1,3
6,axon-configuration-4.10.0.jar,1,41,2,4
7,axon-disruptor-4.10.0.jar,1,22,1,3
8,axon-tracing-opentelemetry-4.10.0.jar,1,5,1,1


### Table 1b - Top 30 artifacts with the highest type count

In [10]:
# Sort by number of types descending
artifacts.sort_values(by=['types','artifactName'], ascending=[False, True]).reset_index(drop=True).head(30)

,artifactName,packages,types,incomingDependencies,outgoingDependencies
0,axon-messaging-4.10.0.jar,64,787,8,0
1,axon-modelling-4.10.0.jar,10,156,6,1
2,axon-eventsourcing-4.10.0.jar,9,133,5,2
3,axon-server-connector-4.10.0.jar,11,131,1,4
4,axon-test-4.10.0.jar,8,87,1,3
5,axon-spring-boot-autoconfigure-4.10.0.jar,9,74,0,7
6,axon-configuration-4.10.0.jar,1,41,2,4
7,axon-disruptor-4.10.0.jar,1,22,1,3
8,axon-tracing-opentelemetry-4.10.0.jar,1,5,1,1


### Table 1c - Top 30 artifacts with the highest number of incoming dependencies

The following table lists the top 30 artifacts that are used the most by other artifacts (highest count of incoming dependencies, highest in-degree).

In [11]:
# Sort by number of incoming dependencies descending
artifacts.sort_values(by=['incomingDependencies','artifactName'], ascending=[False, True]).reset_index(drop=True).head(30)

,artifactName,packages,types,incomingDependencies,outgoingDependencies
0,axon-messaging-4.10.0.jar,64,787,8,0
1,axon-modelling-4.10.0.jar,10,156,6,1
2,axon-eventsourcing-4.10.0.jar,9,133,5,2
3,axon-configuration-4.10.0.jar,1,41,2,4
4,axon-disruptor-4.10.0.jar,1,22,1,3
5,axon-server-connector-4.10.0.jar,11,131,1,4
6,axon-test-4.10.0.jar,8,87,1,3
7,axon-tracing-opentelemetry-4.10.0.jar,1,5,1,1
8,axon-spring-boot-autoconfigure-4.10.0.jar,9,74,0,7


### Table 1d - Top 30 artifacts with the highest number of outgoing dependencies

The following table lists the top 30 artifacts that are depending on the highest number of other artifacts (highest count of outgoing dependencies, highest out-degree).

In [12]:
# Sort by number of outgoing dependencies descending
artifacts.sort_values(by=['outgoingDependencies','artifactName'], ascending=[False, True]).reset_index(drop=True).head(30)

,artifactName,packages,types,incomingDependencies,outgoingDependencies
0,axon-spring-boot-autoconfigure-4.10.0.jar,9,74,0,7
1,axon-configuration-4.10.0.jar,1,41,2,4
2,axon-server-connector-4.10.0.jar,11,131,1,4
3,axon-disruptor-4.10.0.jar,1,22,1,3
4,axon-test-4.10.0.jar,8,87,1,3
5,axon-eventsourcing-4.10.0.jar,9,133,5,2
6,axon-modelling-4.10.0.jar,10,156,6,1
7,axon-tracing-opentelemetry-4.10.0.jar,1,5,1,1
8,axon-messaging-4.10.0.jar,64,787,8,0


### Table 1e - Top 30 artifacts with the lowest package count

In [13]:
# Sort by number of packages ascending
artifacts.sort_values(by=['packages','artifactName'], ascending=[True, True]).reset_index(drop=True).head(30)

,artifactName,packages,types,incomingDependencies,outgoingDependencies
0,axon-configuration-4.10.0.jar,1,41,2,4
1,axon-disruptor-4.10.0.jar,1,22,1,3
2,axon-tracing-opentelemetry-4.10.0.jar,1,5,1,1
3,axon-test-4.10.0.jar,8,87,1,3
4,axon-eventsourcing-4.10.0.jar,9,133,5,2
5,axon-spring-boot-autoconfigure-4.10.0.jar,9,74,0,7
6,axon-modelling-4.10.0.jar,10,156,6,1
7,axon-server-connector-4.10.0.jar,11,131,1,4
8,axon-messaging-4.10.0.jar,64,787,8,0


### Table 1f - Top 30 artifacts with the lowest type count

In [14]:
# Sort by number of types ascending
artifacts.sort_values(by=['types','artifactName'], ascending=[True, True]).reset_index(drop=True).head(30)

,artifactName,packages,types,incomingDependencies,outgoingDependencies
0,axon-tracing-opentelemetry-4.10.0.jar,1,5,1,1
1,axon-disruptor-4.10.0.jar,1,22,1,3
2,axon-configuration-4.10.0.jar,1,41,2,4
3,axon-spring-boot-autoconfigure-4.10.0.jar,9,74,0,7
4,axon-test-4.10.0.jar,8,87,1,3
5,axon-server-connector-4.10.0.jar,11,131,1,4
6,axon-eventsourcing-4.10.0.jar,9,133,5,2
7,axon-modelling-4.10.0.jar,10,156,6,1
8,axon-messaging-4.10.0.jar,64,787,8,0


### Table 1g - Top 30 artifacts with the lowest number of incoming dependencies

The following table lists the top 30 artifacts that are used the least by other artifacts (lowest count of incoming dependencies, lowest in-degree).

In [15]:
# Sort by number of incoming dependencies ascending
artifacts.sort_values(by=['incomingDependencies','artifactName'], ascending=[True, True]).reset_index(drop=True).head(30)

,artifactName,packages,types,incomingDependencies,outgoingDependencies
0,axon-spring-boot-autoconfigure-4.10.0.jar,9,74,0,7
1,axon-disruptor-4.10.0.jar,1,22,1,3
2,axon-server-connector-4.10.0.jar,11,131,1,4
3,axon-test-4.10.0.jar,8,87,1,3
4,axon-tracing-opentelemetry-4.10.0.jar,1,5,1,1
5,axon-configuration-4.10.0.jar,1,41,2,4
6,axon-eventsourcing-4.10.0.jar,9,133,5,2
7,axon-modelling-4.10.0.jar,10,156,6,1
8,axon-messaging-4.10.0.jar,64,787,8,0


### Table 1h - Top 30 artifacts with the lowest number of outgoing dependencies

The following table lists the top 30 artifacts that are depending on the lowest number of other artifacts (lowest count of outgoing dependencies, lowest out-degree).

In [16]:
# Sort by number of outgoing dependencies ascending
artifacts.sort_values(by=['outgoingDependencies','artifactName'], ascending=[True, True]).reset_index(drop=True).head(30)

,artifactName,packages,types,incomingDependencies,outgoingDependencies
0,axon-messaging-4.10.0.jar,64,787,8,0
1,axon-modelling-4.10.0.jar,10,156,6,1
2,axon-tracing-opentelemetry-4.10.0.jar,1,5,1,1
3,axon-eventsourcing-4.10.0.jar,9,133,5,2
4,axon-disruptor-4.10.0.jar,1,22,1,3
5,axon-test-4.10.0.jar,8,87,1,3
6,axon-configuration-4.10.0.jar,1,41,2,4
7,axon-server-connector-4.10.0.jar,11,131,1,4
8,axon-spring-boot-autoconfigure-4.10.0.jar,9,74,0,7


## Cyclic Dependencies

Cyclic dependencies occur when one package uses a class of another package and vice versa. 
These dependencies can lead to problems when one of these packages needs to be changed.

## Table 2a - Cyclic Dependencies Overview

Show the top 40 cyclic dependencies sorted by the most promising to resolve first. This is done by calculating the number of forward dependencies (first cycle participant to second cycle participant) in relation to backward dependencies (second cycle participant back to first cycle participant). The higher this rate (approaching 1), the easier it should be to resolve the cycle by focussing on the few backward dependencies.

Only the top 40 entries are shown. The whole table can be found in the following CSV report:  
`Cyclic_Dependencies`

**Columns:**
- *artifactName* identifies the artifact of the first participant of the cycle
- *packageName* identifies the package of the first participant of the cycle
- *dependentArtifactName* identifies the artifact of the second participant of the cycle
- *dependentPackageName* identifies the package of the second participant of the cycle
- *forwardToBackwardBalance* is between 0 and 1. High for many forward and few backward dependencies.
- *numberForward* contains the number of dependencies from the first participant of the cycle to the second one
- *numberBackward* contains the number of dependencies from the second participant of the cycle back to the first one
- *someForwardDependencies* lists some forward dependencies in the text format "type1 -> type2"
- *backwardDependencies* lists the backward dependencies in the format "type1 <- type2" that are recommended to get resolved

In [17]:
cyclic_dependencies = query_cypher_to_data_frame("../cypher/Cyclic_Dependencies/Cyclic_Dependencies.cypher")
cyclic_dependencies.head(40)

,artifactName,packageName,dependentArtifactName,dependentPackageName,forwardToBackwardBalance,numberForward,numberBackward,someForwardDependencies,backwardDependencies
0,axon-messaging-4.10.0,org.axonframework.eventhandling,axon-messaging-4.10.0,org.axonframework.tracing,0.900000,19,1,"[AbstractEventBus$Builder->NoOpSpanFactory, Ab...",[NestingSpanFactory->EventMessage]
1,axon-messaging-4.10.0,org.axonframework.queryhandling,axon-messaging-4.10.0,org.axonframework.messaging.responsetypes,0.882353,16,1,[SimpleQueryUpdateEmitter->MultipleInstancesRe...,[ConvertingResponseMessage->QueryResponseMessage]
2,axon-messaging-4.10.0,org.axonframework.queryhandling,axon-messaging-4.10.0,org.axonframework.tracing,0.875000,15,1,[DefaultQueryUpdateEmitterSpanFactory$Builder-...,[SpanUtils->QueryMessage]
3,axon-messaging-4.10.0,org.axonframework.eventhandling,axon-messaging-4.10.0,org.axonframework.messaging,0.857143,39,3,[TrackingTokenParameterResolverFactory$Trackin...,"[Headers->EventMessage, Headers->DomainEventMe..."
4,axon-messaging-4.10.0,org.axonframework.eventhandling,axon-messaging-4.10.0,org.axonframework.messaging.annotation,0.840000,23,2,[TimestampParameterResolverFactory->ParameterR...,[SourceIdParameterResolverFactory$SourceIdPara...
5,axon-messaging-4.10.0,org.axonframework.deadline,axon-messaging-4.10.0,org.axonframework.tracing,0.800000,9,1,[SimpleDeadlineManager$Builder->NoOpSpanFactor...,[SpanUtils->DeadlineMessage]
6,axon-messaging-4.10.0,org.axonframework.commandhandling,axon-messaging-4.10.0,org.axonframework.tracing,0.777778,8,1,"[DefaultCommandBusSpanFactory->SpanFactory, De...",[SpanUtils->CommandMessage]
7,axon-eventsourcing-4.10.0,org.axonframework.eventsourcing,axon-eventsourcing-4.10.0,org.axonframework.eventsourcing.eventstore,0.777778,16,2,"[EventSourcingRepository->EventStore, EventSou...","[DomainEventStream->EventStreamUtils, Abstract..."
8,axon-messaging-4.10.0,org.axonframework.commandhandling.callbacks,axon-messaging-4.10.0,org.axonframework.commandhandling,0.733333,13,2,"[NoOpCallback->CommandMessage, NoOpCallback->C...","[SimpleCommandBus$Builder->LoggingCallback, Si..."
9,axon-server-connector-4.10.0,org.axonframework.axonserver.connector.util,axon-server-connector-4.10.0,org.axonframework.axonserver.connector,0.666667,5,1,"[PriorityExecutorService->PriorityCallable, Pr...",[ErrorCode->ExceptionSerializer]


### Table 2b - Cyclic Dependencies Break Down

Lists packages with cyclic dependencies with every dependency in a separate row sorted by the most promising  dependency first.

Only the top 40 entries are shown. The whole table can be found in the following CSV report:  
`Cyclic_Dependencies_Breakdown`

**Columns in addition to Table 2a:**
- *dependency* shows the cycle dependency in the text format "type1 -> type2" (forward) or "type2<-type1" (backward)

In [18]:
cyclic_dependencies_breakdown = query_cypher_to_data_frame("../cypher/Cyclic_Dependencies/Cyclic_Dependencies_Breakdown.cypher",limit=40)
cyclic_dependencies_breakdown

,artifactName,packageName,dependentArtifactName,dependentPackageName,dependency,forwardToBackwardBalance,numberForward,numberBackward
0,axon-messaging-4.10.0,org.axonframework.eventhandling,axon-messaging-4.10.0,org.axonframework.tracing,AbstractEventBus->Span,0.900000,19,1
1,axon-messaging-4.10.0,org.axonframework.eventhandling,axon-messaging-4.10.0,org.axonframework.tracing,SimpleEventBus$Builder->SpanFactory,0.900000,19,1
2,axon-messaging-4.10.0,org.axonframework.eventhandling,axon-messaging-4.10.0,org.axonframework.tracing,DefaultEventBusSpanFactory->SpanFactory,0.900000,19,1
3,axon-messaging-4.10.0,org.axonframework.eventhandling,axon-messaging-4.10.0,org.axonframework.tracing,EventBusSpanFactory->Span,0.900000,19,1
4,axon-messaging-4.10.0,org.axonframework.eventhandling,axon-messaging-4.10.0,org.axonframework.tracing,EventMessage<-NestingSpanFactory,0.900000,19,1
5,axon-messaging-4.10.0,org.axonframework.eventhandling,axon-messaging-4.10.0,org.axonframework.tracing,DefaultEventProcessorSpanFactory->SpanFactory,0.900000,19,1
6,axon-messaging-4.10.0,org.axonframework.eventhandling,axon-messaging-4.10.0,org.axonframework.tracing,DefaultEventBusSpanFactory$Builder->SpanFactory,0.900000,19,1
7,axon-messaging-4.10.0,org.axonframework.eventhandling,axon-messaging-4.10.0,org.axonframework.tracing,DefaultEventBusSpanFactory->Span,0.900000,19,1
8,axon-messaging-4.10.0,org.axonframework.eventhandling,axon-messaging-4.10.0,org.axonframework.tracing,SubscribingEventProcessor$Builder->SpanFactory,0.900000,19,1
9,axon-messaging-4.10.0,org.axonframework.eventhandling,axon-messaging-4.10.0,org.axonframework.tracing,DefaultEventProcessorSpanFactory->Span,0.900000,19,1


### Table 2c - Cyclic Dependencies Break Down - Backward Dependencies Only

Lists packages with cyclic dependencies with every dependency in a separate row sorted by the most promising  dependency first. This table only contains the backward dependencies from the second participant of the cycle back to the first one that are the most promising to resolve.

Only the top 40 entries are shown. The whole table can be found in the following CSV report:  
`Cyclic_Dependencies_Breakdown_BackwardOnly`

In [19]:
cyclic_dependencies_breakdown_backward = query_cypher_to_data_frame("../cypher/Cyclic_Dependencies/Cyclic_Dependencies_Breakdown_Backward_Only.cypher",limit=40)
cyclic_dependencies_breakdown_backward

,artifactName,packageName,dependentArtifactName,dependentPackageName,dependency,forwardToBackwardBalance,numberForward,numberBackward
0,axon-messaging-4.10.0,org.axonframework.eventhandling,axon-messaging-4.10.0,org.axonframework.tracing,EventMessage<-NestingSpanFactory,0.900000,19,1
1,axon-messaging-4.10.0,org.axonframework.queryhandling,axon-messaging-4.10.0,org.axonframework.messaging.responsetypes,QueryResponseMessage<-ConvertingResponseMessage,0.882353,16,1
2,axon-messaging-4.10.0,org.axonframework.queryhandling,axon-messaging-4.10.0,org.axonframework.tracing,QueryMessage<-SpanUtils,0.875000,15,1
3,axon-messaging-4.10.0,org.axonframework.eventhandling,axon-messaging-4.10.0,org.axonframework.messaging,DomainEventMessage<-Headers,0.857143,39,3
4,axon-messaging-4.10.0,org.axonframework.eventhandling,axon-messaging-4.10.0,org.axonframework.messaging,EventMessage<-Headers,0.857143,39,3
5,axon-messaging-4.10.0,org.axonframework.eventhandling,axon-messaging-4.10.0,org.axonframework.messaging,TrackingToken<-StreamableMessageSource,0.857143,39,3
6,axon-messaging-4.10.0,org.axonframework.eventhandling,axon-messaging-4.10.0,org.axonframework.messaging.annotation,DomainEventMessage<-AggregateTypeParameterReso...,0.840000,23,2
7,axon-messaging-4.10.0,org.axonframework.eventhandling,axon-messaging-4.10.0,org.axonframework.messaging.annotation,DomainEventMessage<-SourceIdParameterResolverF...,0.840000,23,2
8,axon-messaging-4.10.0,org.axonframework.deadline,axon-messaging-4.10.0,org.axonframework.tracing,DeadlineMessage<-SpanUtils,0.800000,9,1
9,axon-messaging-4.10.0,org.axonframework.commandhandling,axon-messaging-4.10.0,org.axonframework.tracing,CommandMessage<-SpanUtils,0.777778,8,1


## Interface Segregation Candidates

Well known from [Design Principles and Design Patterns by Robert C. Martin](http://staff.cs.utu.fi/~jounsmed/doos_06/material/DesignPrinciplesAndPatterns.pdf), the *Interface Segregation Principle* suggests that software components should have narrow, focused interfaces rather than large, general-purpose ones. The goal is to minimize the dependencies between components and increase modularity, flexibility, and maintainability.

Smaller, focused and purpose-driven interfaces

- make it easier to modify individual components without affecting the rest of the system.
- make it clearer which client is affected by which change.
- don’t force their clients to depend on methods they don’t need.
- reduce the scope of changes since a change to one component doesn’t affect others.
- lead to a more loosely coupled architecture that is easier to understand and maintain.

Reference: [Analyze java package metrics in a graph database](https://joht.github.io/johtizen/data/2023/04/21/java-package-metrics-analysis.html#interface-segregation)

### How to apply the results

If just one method of a type is used, especially in many places, then the result of this method can be used to call e.g. a method or constuct an object instead of using the whole object and then just calling that single method.

If there are a couple of methods that are used for a distinct purpose, those could be factored out into a separate interface. The original type can extended/implement the new interface so that there are no breaking changes. Then all the callers, that use only this group of methods, can be changed to the new interface.


### Table 4 - Top 40 most used combinations of methods

The following table shows the top 40 most used combinations of methods of larger types that might benefit from applying the *Interface Segregation Principle*. The whole table can be found in the CSV report `Candidates_for_Interface_Segregation`.

In [20]:
interface_segregation_candidates=query_cypher_to_data_frame("../cypher/Internal_Dependencies/Candidates_for_Interface_Segregation.cypher", limit=40)
interface_segregation_candidates

,fullDependentTypeName,declaredMethods,calledMethodNames,calledMethods,callerTypes
0,org.axonframework.commandhandling.CommandMessage,9,[getCommandName],1,20
1,org.axonframework.eventhandling.EventMessage,9,"[getTimestamp, getIdentifier]",2,11
2,org.axonframework.eventhandling.DomainEventMes...,10,[getSequenceNumber],1,9
3,org.axonframework.eventhandling.EventMessage,9,[getIdentifier],1,9
4,org.axonframework.eventhandling.TrackedEventMe...,10,[trackingToken],1,8
5,org.axonframework.commandhandling.GenericComma...,14,[asCommandResultMessage],1,6
6,org.axonframework.eventhandling.DomainEventMes...,10,"[getType, getSequenceNumber, getAggregateIdent...",3,6
7,org.axonframework.messaging.ResultMessage,9,"[exceptionResult, isExceptional]",2,6
8,org.axonframework.eventhandling.ReplayToken,13,[createReplayToken],1,5
9,org.axonframework.deadline.GenericDeadlineMessage,11,[asDeadlineMessage],1,5


## Package Usage

### Table 5 - Types that are used by multiple packages

This table shows the top 40 packages that are used by the highest number of different packages. The whole table can be found in the CSV report `List_types_that_are_used_by_many_different_packages`.


In [21]:
types_used_by_many_packages=query_cypher_to_data_frame("../cypher/Internal_Dependencies/List_types_that_are_used_by_many_different_packages.cypher", limit=40)
types_used_by_many_packages

,fullQualifiedDependentTypeName,dependentTypeName,dependentTypeLabels,numberOfUsingPackages
0,org.axonframework.common.BuilderUtils,BuilderUtils,"[Type, File, Java, ByteCode, Class, Mark4TopCe...",49
1,org.axonframework.common.AxonConfigurationExce...,AxonConfigurationException,"[Type, File, Java, ByteCode, Class, Mark4TopCe...",42
2,org.axonframework.messaging.Message,Message,"[Type, File, Java, ByteCode, GenericDeclaratio...",41
3,org.axonframework.messaging.MetaData,MetaData,"[Type, File, Java, ByteCode, Class, Mark4TopCe...",39
4,org.axonframework.serialization.Serializer,Serializer,"[Type, File, Java, ByteCode, Interface, Mark4T...",36
5,org.axonframework.eventhandling.EventMessage,EventMessage,"[Type, File, Java, ByteCode, GenericDeclaratio...",35
6,org.axonframework.messaging.unitofwork.UnitOfWork,UnitOfWork,"[Type, File, Java, ByteCode, GenericDeclaratio...",32
7,org.axonframework.common.transaction.Transacti...,TransactionManager,"[Type, File, Java, ByteCode, Interface, Mark4T...",31
8,org.axonframework.common.Assert,Assert,"[Type, File, Java, ByteCode, Class, Mark4TopCe...",29
9,org.axonframework.serialization.SerializedObject,SerializedObject,"[Type, File, Java, ByteCode, GenericDeclaratio...",27


### Table 6 - Packages that are used by multiple artifacts

This table shows the top 30 artifacts that only use a few (compared to all existing) packages of another artifact.
The whole table can be found in the CSV report `ArtifactPackageUsage`.

In [22]:
used_packages_of_dependent_artifact=query_cypher_to_data_frame("../cypher/Internal_Dependencies/How_many_packages_compared_to_all_existing_are_used_by_dependent_artifacts.cypher",limit=30)
used_packages_of_dependent_artifact

,artifactName,dependentArtifactName,dependentPackages,dependentArtifactPackages,packageUsagePercentage,dependentFullQualifiedPackageNames,dependentPackageNames
0,axon-modelling-4.10.0,axon-modelling-4.10.0,6,10,0.600000,"[org.axonframework.modelling.saga, org.axonfra...","[saga, jpa, repository, inspection, command, m..."
1,axon-test-4.10.0,axon-test-4.10.0,5,8,0.625000,"[org.axonframework.test, org.axonframework.tes...","[test, deadline, matchers, utils, eventscheduler]"
2,axon-messaging-4.10.0,axon-messaging-4.10.0,41,64,0.640625,"[org.axonframework.serialization, org.axonfram...","[serialization, common, eventhandling, transac..."
3,axon-eventsourcing-4.10.0,axon-eventsourcing-4.10.0,7,9,0.777778,[org.axonframework.eventsourcing.eventstore.jd...,"[jdbc, eventstore, snapshotting, statements, j..."
4,axon-server-connector-4.10.0,axon-server-connector-4.10.0,9,11,0.818182,"[org.axonframework.axonserver.connector, org.a...","[connector, util, heartbeat, query, axon, comm..."
5,axon-spring-boot-autoconfigure-4.10.0,axon-spring-boot-autoconfigure-4.10.0,8,9,0.888889,"[org.axonframework.springboot.util, org.axonfr...","[util, connection, jpa, springboot, axonserver..."


### Table 7 - Types that are used by multiple artifacts

This table shows the top 30 types that only use a few (compared to all existing) types of another artifact. The whole table can be found in the CSV report `ClassesPerPackageUsageAcrossArtifacts`.

In [23]:
used_types_of_dependent_artifact=query_cypher_to_data_frame("../cypher/Internal_Dependencies/How_many_classes_compared_to_all_existing_in_the_same_package_are_used_by_dependent_packages_across_different_artifacts.cypher", limit=30)
used_types_of_dependent_artifact

,artifactName,dependentArtifactName,packageName,dependentPackage.fqn,dependentTypes,dependentPackageTypes,typeUsagePercentage,dependentTypeNames
0,axon-spring-boot-autoconfigure-4.10.0,axon-messaging-4.10.0,org.axonframework.springboot.util,org.axonframework.eventhandling,1,100,0.010000,[org.axonframework.eventhandling.EventMessage]
1,axon-server-connector-4.10.0,axon-messaging-4.10.0,org.axonframework.axonserver.connector,org.axonframework.eventhandling,1,100,0.010000,[org.axonframework.eventhandling.EventBusSpanF...
2,axon-modelling-4.10.0,axon-messaging-4.10.0,org.axonframework.modelling.saga.metamodel,org.axonframework.eventhandling,1,100,0.010000,[org.axonframework.eventhandling.EventMessage]
3,axon-spring-boot-autoconfigure-4.10.0,axon-messaging-4.10.0,org.axonframework.springboot.autoconfig.legacyjpa,org.axonframework.eventhandling,1,100,0.010000,[org.axonframework.eventhandling.EventBus]
4,axon-test-4.10.0,axon-messaging-4.10.0,org.axonframework.test.matchers,org.axonframework.eventhandling,1,100,0.010000,[org.axonframework.eventhandling.EventMessage]
5,axon-eventsourcing-4.10.0,axon-messaging-4.10.0,org.axonframework.eventsourcing.snapshotting,org.axonframework.eventhandling,1,100,0.010000,[org.axonframework.eventhandling.DomainEventData]
6,axon-eventsourcing-4.10.0,axon-modelling-4.10.0,org.axonframework.eventsourcing.conflictresolu...,org.axonframework.modelling.command,1,54,0.018519,[org.axonframework.modelling.command.Conflicti...
7,axon-eventsourcing-4.10.0,axon-modelling-4.10.0,org.axonframework.eventsourcing.eventstore.jdbc,org.axonframework.modelling.command,1,54,0.018519,[org.axonframework.modelling.command.Concurren...
8,axon-server-connector-4.10.0,axon-modelling-4.10.0,org.axonframework.axonserver.connector,org.axonframework.modelling.command,1,54,0.018519,[org.axonframework.modelling.command.Concurren...
9,axon-eventsourcing-4.10.0,axon-messaging-4.10.0,org.axonframework.eventsourcing.conflictresolu...,org.axonframework.eventhandling,2,100,0.020000,[org.axonframework.eventhandling.DomainEventMe...


### Table 8 - Duplicate package names across artifacts

This table shows the top 30 duplicate package names across artifacts. They are ordered by the number of duplicates descending.

This might lead to confusion, makes importing more error prone and might even lead to duplicate classes where only one of them will be loaded by the class loader. If a package is named the same way in two or more artifacts this even allows another artifact to access package protected classes, methods or members which might not be intended. 

The whole table can be found in the CSV report `DuplicatePackageNamesAcrossArtifacts`.

In [24]:
duplicate_package_names_across_artifacts=query_cypher_to_data_frame("../cypher/Artifact_Dependencies/Artifacts_with_duplicate_packages.cypher", limit=30)
duplicate_package_names_across_artifacts

,packageName,duplicates,artifactNames


### Table 9 - Annotated elements

This table shows 30 most used Java Annotations including some examples where they are used.


In [25]:
annotated_elements=query_cypher_to_data_frame("../cypher/Java/Annotated_code_elements.cypher", limit=30)
annotated_elements

,annotationName,languageElement,numberOfAnnotatedElements,examples
0,javax.annotation.Nonnull,Parameter,1615,[org.axonframework.test.aggregate.ResultValida...
1,java.lang.Deprecated,Method,130,[org.axonframework.test.aggregate.ResultValida...
2,org.springframework.context.annotation.Bean,Method,110,[org.axonframework.springboot.autoconfig.AxonS...
3,org.springframework.boot.autoconfigure.conditi...,Method,76,[org.axonframework.springboot.autoconfig.AxonS...
4,java.lang.FunctionalInterface,Interface,64,"[org.axonframework.test.matchers.FieldFilter, ..."
5,javax.annotation.Nullable,Parameter,61,[org.axonframework.modelling.command.CreationP...
6,javax.annotation.Nonnull,Method,50,[org.axonframework.test.aggregate.AggregateTes...
7,java.lang.annotation.Target,Annotation,43,[org.axonframework.modelling.command.Aggregate...
8,java.lang.annotation.Retention,Annotation,43,[org.axonframework.modelling.command.Aggregate...
9,java.lang.Deprecated,Class,43,[org.axonframework.test.aggregate.StubAggregat...


### Table 10 - Distance distribution between dependent files

This table shows the file directory distance distribution between dependent files. Intuitively, the distance is given by the fewest number of change directory commands needed to navigate between a file and a dependency it uses. Those are aggregate to see how many dependent files are in the same directory, how many are just one change directory command apart, and so on.

In [26]:
query_first_non_empty_cypher_to_data_frame("../cypher/Internal_Dependencies/Get_file_distance_as_shortest_contains_path_for_dependencies.cypher",
                                           "../cypher/Internal_Dependencies/Set_file_distance_as_shortest_contains_path_for_dependencies.cypher", limit=20)

,dependency.fileDistanceAsFewestChangeDirectoryCommands,numberOfDependencies,numberOfDependencyUsers,numberOfDependencyProviders,examples
0,0,2597,1025,1157,[/axon-spring-boot-autoconfigure-4.10.0.jar us...
1,1,90,82,38,[/org/axonframework/test/deadline uses /org/ax...
2,2,2499,759,470,[/org/axonframework/test/aggregate/ResultValid...
3,4,2223,401,465,[/org/axonframework/springboot/service/connect...
